# guiview.py

Utility to **view** and **output** *videos* + *supplementary files*.

In this workspace, these videos were recorded with *guirecord.py* and each **video (.avi)** has a **timelog (.txt)** and **metalog (.metalog)** associated with it. And when outputting an edited video, these files are also output [and modified] for the [modified] output video.

Output is a new set of three files named: videofn.procN.ext  

 * videofn - name of input w/o ext
 * N - number to make file unqiue in that directory (guid-ish)
 * ext - the respective extension (.avi, .txt, .metalog)

This utility is essentially a CRUD-app for the metalog file associated with the file. We write information about what is happening in the frame, and the "score" which is position of the object(s). We don't modify the actual video data itself, except to include / exclude frames, and to output a video with a [possibly different] encoding.

### Quickstart:

` > python guiview.py --file data/proc/tmp/demo-agenda/output4.proc2.avi --showscoring`

` > python guiview.py --dir data/proc/raw/oct20/ --firstN 20`

### Overview:
<img src="img/guiview_instructions_1.png">

### Commands 

Command Line Options
***
* --file [path-fn]:    open vid file at path-fn; play at normal speed
* --dir [dir-path]:    walk through files in dir-path, play as fast as possible; rewind not possible
* --firstN [int]:      for --dir runs, only play first N frames; then goes to next video
* --showscoring:       if used, shows any scoring entries in metalog on main_display and score_display
* --scoringenum [int]: when there are multiple objects, this decides which object to focus on in score_display
* --track:             turn on Track module from the start
* --tracktimer:        if used, populate 'track timer' on gui with track-calc-time for each frame


Major Gui Operations
***
| gui name        | performs                                                   |
|:----------------|:-----------------------------------------------------------|
|Play             | play video                                                 
|Advance          | advance one frame 
|retreat          | rewind one frame 
|Init output      | create new vid + supl-file outputs at that directory       
|h264/lossless    | encoding for output video
|Writevid         | when on (button is green) every time you advance, a frame is written to vid, and non-scoring params to metalog
|writeFrame       | output current frame + non-scoring params to metalog; advance to next frame
|writef+scorinG   | output current frame + scoring-params + non-scoring params to metalog; advance to next frame
|writef+overide   | use notes/framenote-override.json to update non-scoring params; write out to vid and metalog
|selectZoom       | draw zoom roi onto main_display; then hit Enter
|roi main         | draw obj roi onto main_display; then hit Enter; will use 'score type' and 'obj enum' selected
|roi zoom         | draw obj roi onto zoom_display; then hit Enter; will use 'score type' and 'obj enum' selected
|reset            | deletes all current obj roi's
|obj enum         | enum for obj
|score type       | if ray, do two selectROI's hitting enter after each one; ray is line between two center points of two roi's
|Track            | if on, try to do tracking on frame


Keyboard Shortcuts
***
Any capital letters on gui represent shorcut keys, e.g. 'Advance' is key=A. 

Any letters between &lt;tags&gt; is also a shortcut key, e.g. 'roi main &lt;C&gt;' is key=C. 

Note: you need to have focus on Tkinter-gui for these to work. 

See modules/GuiC.py cmd_keypress()

### Rationale

Designed to work as unix-like utility for videos: everything is a file (video + metadata) and tool can be invoked from command line. So we can use file names and directory hierarchies to organize data. And diffs to examine changes.

But also has strong graphical editing properties, as we'll need to magnify image and select regions.

The tool can be used to quickly evaluate performance on multiple videos, or drill down into particular frame-regions of one video.


### Associated Files
* **metalog** files are json files with properties about:

 * what parameters were used to record the video:
    * "cam_num":0
      
      ...
      
 * the conditions under which the video was recorded:
    * "light_level" : "sunlight_strong"
    * "ball_color" : "green"
    * these are useful for categorizing videos into evaluation groups
      
      ...
     
  * if processed ("processed":true) - meaning edited by guiview - a framenote for each frame written to output video:
  
    * parameters about the current scene:
      * "ball_present" :  true

      ...
      
    * obj scoring at the current frame:
      * "scoring" :  {"0":{"type":"circle", "data":[100,100,50,50]}}
      
      ...

* **timelog** files are space separated text files. + each line represents the read time for that loop

  see <a src="timelog.py.ipynb">timelog.ipynb</a> for further description analysis.
  
  see books/sync-frame-to-framelog.ipynb for description of lagging frame


### CRUD Frame-Score

* Create Score "hand drawn score"
* Read Score "--showscoring"
* Update Score "hand drawn score"
* Delete Score - edit the raw metalog json


### CRUD Frame-Params
* Create Param: occurs first time you process a video. the params in notes/guiview.jsonc will be written into the framenote
* Read Param: occurs in NotesFactory, handled throught program
* Update Param:
  * single frame overwrite in notes/framenotes.json
  * batch frame overwrite in notes/framenote-override.jsonc
    note: can only update exisitng params, not add new ones. maybe this will change
* Delete Param: edit the raw metalog json

### Workflow


To process a raw video ("processed": false), first check its metalog is correct as this will carry over to new files.

Now focus on guiview.jsonc and initialize it to default values for your framenotes 

Find the frame where you want to start the video, usually a little past the beginning to avoid setup time

setting window position for subsequent runs